In [ ]:
"""
network_analysis_examples.ipynb

Created on Sun Mar 19 2023

@author: Lukas

This notebook serves as a playground for the code in the
network analysis repo.
"""

In [ ]:
# import packages

import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# create an empty pandas dataframe with columns "Name", "Type", "Suppliers", and "Shareholders"

df = pd.DataFrame(columns=["Name", "Type", "Suppliers", "Shareholders"])


# populate the dataframe

df = df.append({"Name": "Company A", "Type": "Company", "Suppliers": ["Company B", "Company C"], "Shareholders": ["Bank A", "Individual A"]}, ignore_index=True)
df = df.append({"Name": "Company B", "Type": "Company", "Suppliers": ["Company C", "Company D"], "Shareholders": ["Bank A", "Individual B"]}, ignore_index=True)
df = df.append({"Name": "Company C", "Type": "Company", "Suppliers": ["Company D"], "Shareholders": ["Bank B", "Individual A"]}, ignore_index=True)
df = df.append({"Name": "Company D", "Type": "Company", "Suppliers": [], "Shareholders": ["Bank B", "Individual C"]}, ignore_index=True)

In [ ]:
# create a directed graph of suppliers from the dataframe

G = nx.DiGraph()

for index, row in df.iterrows():
    for supplier in row["Suppliers"]:
        G.add_edge(supplier, row["Name"])

In [ ]:
# draw the graph, move the labels above the nodes so they don't overlap

pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_size=500, node_color="orange")
labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels, label_pos=0.3, font_size=8)
plt.show()

In [ ]:
# create a directed graph of shareholders from the dataframe

H = nx.DiGraph()

for index, row in df.iterrows():
    for shareholder in row["Shareholders"]:
        H.add_edge(shareholder, row["Name"])

In [ ]:
# draw the graph, extend pos to include the new nodes

pos.update(nx.spring_layout(H))

# color the nodes according to their type

node_colors = []
for node in H.nodes():
    if node in df["Name"].values:
        node_colors.append("orange")
    else:
        node_colors.append("lightblue")

# draw the graph

nx.draw(H, pos, with_labels=True, node_size=500, node_color=node_colors)
labels = nx.get_edge_attributes(H, 'weight')
nx.draw_networkx_edge_labels(H, pos, edge_labels=labels, label_pos=0.3, font_size=8)
plt.show()

In [ ]:
# create a heterogeneous graph from the two directed graphs

I = nx.compose(G, H)

# color the edges according to the network they belong to

edge_colors = []
for edge in I.edges():
    if edge in G.edges():
        edge_colors.append("red")
    else:
        edge_colors.append("blue")

# color the nodes according to their type

node_colors = []
for node in I.nodes():
    if node in df["Name"].values:
        node_colors.append("orange")
    else:
        node_colors.append("lightblue")

# creat a new pos, make sure edges don't overlap with nodes or legends

pos = nx.spring_layout(I)
pos.update((n, (x, y+0.1)) for n, (x, y) in pos.items())
pos.update((n, (x, y-0.1)) for n, (x, y) in pos.items())

# draw the graph

nx.draw(I, pos, with_labels=True, node_size=500, node_color=node_colors, edge_color=edge_colors)
labels = nx.get_edge_attributes(I, 'weight')
nx.draw_networkx_edge_labels(I, pos, edge_labels=labels, label_pos=0.3, font_size=8)

# add two legends, one for the edge colors and one for the node colors

edge_legend = plt.legend([plt.Line2D([0], [0], color="red", lw=4), plt.Line2D([0], [0], color="blue", lw=4)], ["Supplier", "Shareholder"], loc="lower left", title="Edge Color")
node_legend = plt.legend([plt.Line2D([0], [0], color="orange", lw=4), plt.Line2D([0], [0], color="lightblue", lw=4)], ["Company", "Bank/Individual"], loc="lower right", title="Node Color")
plt.gca().add_artist(edge_legend)
plt.show()